In [61]:
!pip install --upgrade opentrons
!pip install pandas
!pip install numpy


Requirement already up-to-date: opentrons in c:\users\mkell\anaconda3\lib\site-packages (4.1.1)


ERROR: Could not find a version that satisfies the requirement Tkinter (from versions: none)
ERROR: No matching distribution found for Tkinter


In [1]:
from opentrons import protocol_api
import tkinter as tk #basic python package apparently
import csv
import pandas as pd
import numpy as np
import os

In [2]:
## Meta Data

metadata = {'protocolName': 'RED',
'author':'Mason Kellinger masonkellinger@vt.edu', 'Steven Kenah ksteven18@vt.edu'
 'description': 'Restriction Enzyme Digestion',
 'apiLevel': '2.9'
}
print(metadata)

os.getcwd()

{'protocolName': 'RED', 'author': 'Mason Kellinger masonkellinger@vt.edu', 'Steven Kenah ksteven18@vt.edudescription': 'Restriction Enzyme Digestion', 'apiLevel': '2.9'}


'C:\\Users\\mkell\\Documents\\GitHub\\opentrons\\Restriction'

In [3]:

#path = "/Shared drives/PlantSynBioLab/Opentrons/Restriction Enzyme Digestion/csvfiles"
path = "C:/Users/mkell/Desktop/RestrictionTestCSV2.csv"

config_file = pd.read_csv(path, engine = 'python', encoding='utf-8-sig') ## last 2 parameters are error corrections


config_df = pd.DataFrame(config_file)


config_df

,enzyme,incu_temp,incu_time,inac_temp,amount_dna,conc_dna
0,UNITS:,celsius,minutes,celsius,µg,µg/µL
1,EcoRI,37,10,65,1,1


In [32]:
from tkinter import *

    # setup window
root = Tk()

    # window size 
root.geometry("500x500")
root.title("Enzyme Entry")

    # make text entry box 
entry = Entry(root)
entry.pack()

def enter():
    label1 = Label(root,text = entry.get())
    label1.pack()
   
    # assign global variable to use outside of `enter` function 
    global enzyme_name
    enzyme_name = entry.get()
    
    # display entered value and  
    label2 = Label(root,text =  str(config_file.loc[config_file.enzyme==enzyme_name]))
    label2.place(relx = 0.5, 
                   rely = 0.5,
                   anchor = 'center')
    
    #root.destroy()
    
    
    
def confirm():
    root.destroy()
    
button1 = Button(root,text="Enter your enzyme",command = enter)
button1.pack()

button2 = Button(root,text="Confirm your data, if incorrect fix in table",command = confirm)
button2.place(relx = 0.7, 
                   rely = 0.7,
                   anchor = 'center')

    # startup window loop  
root.mainloop()



In [25]:
# maybe you could input the index of your enzyme as n and then iloc[n,1] ect.


incubation_temp = int(config_file.loc[config_file.enzyme==enzyme_name,"incu_temp"])
incubation_time = int(config_file.loc[config_file.enzyme==enzyme_name,"incu_time"])
inactivation_temp = int(config_file.loc[config_file.enzyme==enzyme_name,"inac_temp"])
amount_of_dna = int(config_file.loc[config_file.enzyme==enzyme_name,"amount_dna"])
concentration_of_dna = int(config_file.loc[config_file.enzyme==enzyme_name,"conc_dna"])

volume_dna = amount_of_dna * (1/concentration_of_dna)
volume_water = 17 - volume_dna



,enzyme,incu_temp,incu_time,inac_temp,amount_dna,conc_dna
1,EcoRI,37,10,65,1,1


In [ ]:
## Configuration
def get_config(config_file):
    



In [ ]:
## protocol

def run(protocol: protocol_api.Context):
    
    ## Loading labware
    TempUnit = protocol.load_module('Temperature Module',3)
    well = TempUnit.load_labware('opentrons_24_aluminumblock_nest_2ml_snapcap')
    
    tiprack1 = protocol.load_labware("opentrons_96_tiprack_10ul",'1')
    tiprack2 = protocol.load_labware("opentrons_96_tiprack_20ul","2")
    tuberack = protocol.load_labware("opentrons_24_tuberack_nest_2ml_snapcap","4")
    
        ## Loading tools
    pipette_left = protocol.load_tools('p20_single','left','tip_racks = [tiprack1]')
    pipette_right = protocol.load_tools('p20_single','right','tip_racks = [tiprack2]')
    
        ## Operations

    ## get tip -> get x amount of water to fill to 20uL -> dispense into tube
    pipette_left.pick_up_tip(tiprack1['A1'])
    pipette_left.aspirate(volume_water,location,2)
    pipette_left.dispense(volume_water,well['A1'],2) #create function to deteremine amount needed
    pipette_left.blow_out()
    pipette_left.drop_tip()



    ## get 2uL of 10x buffer -> dispense into tube -> get rid of tip

    pipette_left.pick_up_tip()
    pipette_left.aspirate(2,location,2)
    pipette_left.dispense(2,well['A1'],2)
    pipette_left.blow_out()
    pipette_left.drop_tip()


    ## get tip -> get y amount of DNA -> dispense into tube -> get rid of tip

    pipette_left.pick_up_tip()
    pipette_left.aspirate(volume_dna,location,2) #create function to determine
    pipette_left.dispense(volume_dna,well['A1'],2)
    pipette_left.drop_tip()
    ## we don't blow out so we dont aerosolize dna

    ## get tip -> get 1uL Enzyme -> dispense into tube -> get rid of tip

    pipette_left.pick_up_tip()
    pipette_left.aspirate(1,location,rate)
    pipette_left.dispense(1,well['A1'],1)
    pipette_left.blow_out()
    pipette_left.drop_tip()
    
    #mix

    pipette_left.mix(3,20,well['A1'])  #repetetions, max volume uL, location
    #this helps rinse the extra enzyme as well

## Temp Unit - incubate at temp for time t and then inactivate at temp for 10 min then rest at 4   degrees
                                        
    TempUnit.set_temperature(incubation_temp)
    protocol.delay(incubation_time)
    TempUnit.set_temperature(inactivation_temp)
    protocol.delay(10)
    TempUnit.set_temperature(4)
    

In [ ]:
def main():
    get_config(config_file)
    run(protocol: protocol_api.Context)